In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/hackerearth-tweet-sentiment/train.csv
/kaggle/input/hackerearth-tweet-sentiment/test.csv


In [26]:
train.head()

,original_text,sentiment_class
0,happy mothersday amazing mothers know hard not...,0
1,happy mothers day mum im sorry cant bring moth...,0
2,happy mothers day mothers days work today quie...,-1
3,happy mothers day beautiful womanroyalty sooth...,0
4,remembering amazing ladies made late grandmoth...,-1


In [27]:
test.head()

,original_text
0,yeah cooked potatoes years old mean threw bag ...
1,happy mothers day mums stepmums grandmothers d...
2,love people uk however awoke saw mothersday tr...
3,happy birthday happy mother ’ day mum ’ see to...
4,happy mothers day wonderful mothers world live...


In [28]:
import gc
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

from transformers import RobertaTokenizer, RobertaConfig, TFRobertaPreTrainedModel
from transformers.modeling_tf_roberta import TFRobertaMainLayer
from transformers.modeling_tf_utils import get_initializer

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [2]:
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
import scikitplot as skplt
from wordcloud import WordCloud

In [3]:
from bs4 import BeautifulSoup
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [4]:
!pip install Keras==2.3

     |████████████████████████████████| 377 kB 4.3 MB/s eta 0:00:01
  Attempting uninstall: Keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
import re
import gc
import os
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.


In [6]:
train = pd.read_csv("/kaggle/input/hackerearth-tweet-sentiment/train.csv")
test = pd.read_csv("/kaggle/input/hackerearth-tweet-sentiment/test.csv")

In [7]:
train.head(2)

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0


In [8]:
len(train['original_text'].unique())

3235

In [9]:
# train.groupby(['lang']).count()
# train['lang'].value_counts()
len(train['lang'].unique())

233

In [10]:
# train.groupby(['retweet_count']).count()
train['retweet_count'].value_counts()

0                2521
1                 291
en                142
2                  84
3                  45
                 ... 
118                 1
96                  1
-0.9278231369       1
28                  1
-0.7229893804       1
Name: retweet_count, Length: 72, dtype: int64

In [11]:
# len(train['original_author'].unique())
train['original_author'].value_counts()

LToddWood          369
0                  146
MoM_Mastering       99
andreaanderegg      86
LiveGoodProduct     71
                  ... 
lynseyapeterson      1
Wardworkz            1
chic_vegan           1
amandabfulton        1
Boggyshaw            1
Name: original_author, Length: 1574, dtype: int64

In [12]:
train[train['original_author']=='0']['sentiment_class'].value_counts()

0    145
1      1
Name: sentiment_class, dtype: int64

In [13]:
len(train['id'].unique())
train.drop(["id","lang","original_author","retweet_count"],axis=1,inplace=True)
test.drop(["id","lang","original_author","retweet_count"],axis=1,inplace=True)

#### Target Class distribution

In [14]:
train['sentiment_class'].value_counts()

 0    1701
-1     769
 1     765
Name: sentiment_class, dtype: int64

In [15]:
train.head(2)

,original_text,sentiment_class
0,Happy #MothersDay to all you amazing mothers o...,0
1,Happy Mothers Day Mum - I'm sorry I can't be t...,0


In [16]:
train.isna().sum()

original_text      0
sentiment_class    0
dtype: int64

In [17]:
train[train['sentiment_class']==1].head()

,original_text,sentiment_class
6,Happy Mother’s Day to all you wonderful mums o...,1
7,Show me a mother of two as sexy as Rosie Happy...,1
8,"To all the mothers, grandmothers, aunts, siste...",1
11,"HAPPY MOTHERS DAY , CHRISTINE | DAILY LIFE For...",1
12,Happy Mother's Day! Check out my latest blog a...,1


In [18]:
train[train['sentiment_class']==0].head()

,original_text,sentiment_class
0,Happy #MothersDay to all you amazing mothers o...,0
1,Happy Mothers Day Mum - I'm sorry I can't be t...,0
3,Happy mothers day to this beautiful woman...ro...,0
5,HAPPY MOTHER’S DAY! BRATZ WORLD FAMILIEZ YASMI...,0
9,"To all the mothers around the World, Gas To He...",0


In [19]:
train[train['sentiment_class']==-1].head()

,original_text,sentiment_class
2,Happy mothers day To all This doing a mothers ...,-1
4,Remembering the 3 most amazing ladies who made...,-1
13,Happy Mothers Day to everyone tuning in. This ...,-1
17,Happy Mothers Day ! We hope your mums aren't p...,-1
29,Happy mothers day to all those mums whos child...,-1


# Text Processing

Punctuations

In [20]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
def punctuation_removal(messy_str):
    clean_list = [char for char in messy_str if char not in string.punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

train['original_text'] = train['original_text'].apply(punctuation_removal)
test['original_text'] = test['original_text'].apply(punctuation_removal)

In [22]:
train['original_text'][6]

'Happy Mother’s Day to all you wonderful mums out there mothersday cupcakes supermums traceyscakecraft couturesugarpaste saracinouk hinckley Traceys CakeCraft httpswww instagramcompBCUzHjgAu igshidlf3elmlv7q7g …'

### StopWords

In [23]:
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
stop = stopwords.words('english')

stop.remove('not')
stop.remove("don't")

stop_words = []

for item in stop: 
    new_item = punctuation_removal(item)
    stop_words.append(new_item) 
    
def stopwords_removal(messy_str):
    messy_str = word_tokenize(messy_str)
    return [word.lower() for word in messy_str 
            if word.lower() not in stop_words ]


train['original_text'] = train['original_text'].apply(stopwords_removal)
test['original_text'] = test['original_text'].apply(stopwords_removal)


### Dropping Numbers

In [24]:
import re
def drop_numbers(list_text):
    list_text_new = []
    for i in list_text:
        if not re.search('\d', i):
            list_text_new.append(i)
    return ' '.join(list_text_new)

train['original_text'] = train['original_text'].apply(drop_numbers)
test['original_text'] = test['original_text'].apply(drop_numbers)

In [25]:
train['original_text']

0       happy mothersday amazing mothers know hard not...
1       happy mothers day mum im sorry cant bring moth...
2       happy mothers day mothers days work today quie...
3       happy mothers day beautiful womanroyalty sooth...
4       remembering amazing ladies made late grandmoth...
                              ...                        
3230    sisters sisters law also friendsthis wishing h...
3231    happy mother ’ day mums step mums aunties gran...
3232    happy mothers day craziest woman know thanks a...
3233    happy mothers day amazing wife love like crazy...
3234    wishing safe happy mothers day mothersday reed...
Name: original_text, Length: 3235, dtype: object

### Spiltting

In [ ]:
X = train.drop('sentiment_class',axis=1)
y = train['sentiment_class']

In [ ]:
train.head(2)

In [ ]:
train['sentiment_class'].value_counts()

In [ ]:
one_hot = pd.get_dummies(train["sentiment_class"])
train.drop(['sentiment_class'],axis=1,inplace=True)
train = pd.concat([train,one_hot],axis=1)
train.head()

In [ ]:
test.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train["original_text"].values, train.drop(['original_text'],axis=1).values, test_size=0.5, random_state=42)

In [ ]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=140)

In [ ]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, maxlen=140)

In [ ]:
X_test

In [ ]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=140))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train,
                    batch_size=1024,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

In [ ]:
score = model.evaluate(X_test, y_test,
                       batch_size=256, verbose=1)
print('Test accuracy:', score[1])

In [ ]:
preds = model.predict(X_test)

In [ ]:
print(classification_report(np.argmax(y_test,axis=1),np.argmax(preds,axis=1)))

In [ ]:
test.shape

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test)
test = pad_sequences(test_sequences, maxlen=140)

In [ ]:
test

In [ ]:
# fastai
from fastai import *
from fastai.text import *
from fastai.callbacks import *

# transformers
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig

from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

In [ ]:
import fastai
import transformers
print('fastai version :', fastai.__version__)
print('transformers version :', transformers.__version__)

In [ ]:
MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)
}

In [ ]:
# Parameters
seed = 42
use_fp16 = False
bs = 16

model_type = 'roberta'
pretrained_model_name = 'roberta-base'

In [ ]:
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]


In [ ]:
# model_class.pretrained_model_archive_map.keys()

In [ ]:
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [ ]:
seed_all(seed)

In [ ]:
class TransformersBaseTokenizer(BaseTokenizer):
    """Wrapper around PreTrainedTokenizer to be compatible with fast.ai"""
    def __init__(self, pretrained_tokenizer: PreTrainedTokenizer, model_type = 'bert', **kwargs):
        self._pretrained_tokenizer = pretrained_tokenizer
        self.max_seq_len = pretrained_tokenizer.max_len
        self.model_type = model_type

    def __call__(self, *args, **kwargs): 
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length and add the spesial tokens"""
        CLS = self._pretrained_tokenizer.cls_token
        SEP = self._pretrained_tokenizer.sep_token
        if self.model_type in ['roberta']:
            tokens = self._pretrained_tokenizer.tokenize(t, add_prefix_space=True)[:self.max_seq_len - 2]
            tokens = [CLS] + tokens + [SEP]
        else:
            tokens = self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2]
            if self.model_type in ['xlnet']:
                tokens = tokens + [SEP] +  [CLS]
            else:
                tokens = [CLS] + tokens + [SEP]
        return tokens

In [ ]:
transformer_tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)
transformer_base_tokenizer = TransformersBaseTokenizer(pretrained_tokenizer = transformer_tokenizer, model_type = model_type)
fastai_tokenizer = Tokenizer(tok_func = transformer_base_tokenizer, pre_rules=[], post_rules=[])

In [ ]:
class TransformersVocab(Vocab):
    def __init__(self, tokenizer: PreTrainedTokenizer):
        super(TransformersVocab, self).__init__(itos = [])
        self.tokenizer = tokenizer
    
    def numericalize(self, t:Collection[str]) -> List[int]:
        "Convert a list of tokens `t` to their ids."
        return self.tokenizer.convert_tokens_to_ids(t)
        #return self.tokenizer.encode(t)

    def textify(self, nums:Collection[int], sep=' ') -> List[str]:
        "Convert a list of `nums` to their tokens."
        nums = np.array(nums).tolist()
        return sep.join(self.tokenizer.convert_ids_to_tokens(nums)) if sep is not None else self.tokenizer.convert_ids_to_tokens(nums)
    
    def __getstate__(self):
        return {'itos':self.itos, 'tokenizer':self.tokenizer}

    def __setstate__(self, state:dict):
        self.itos = state['itos']
        self.tokenizer = state['tokenizer']
        self.stoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.itos)})

In [ ]:
transformer_vocab =  TransformersVocab(tokenizer = transformer_tokenizer)
numericalize_processor = NumericalizeProcessor(vocab=transformer_vocab)

tokenize_processor = TokenizeProcessor(tokenizer=fastai_tokenizer, include_bos=False, include_eos=False)

transformer_processor = [tokenize_processor, numericalize_processor]

In [ ]:
pad_first = bool(model_type in ['xlnet'])
pad_idx = transformer_tokenizer.pad_token_id

In [ ]:
tokens = transformer_tokenizer.tokenize('hello world  its me')
print(tokens)
ids = transformer_tokenizer.convert_tokens_to_ids(tokens)
print(ids)
transformer_tokenizer.convert_ids_to_tokens(ids)

In [ ]:
databunch = (TextList.from_df(train, cols='original_text', processor=transformer_processor)
             .split_by_rand_pct(0.1,seed=seed)
             .label_from_df(cols= 'sentiment_class')
             .add_test(test)
             .databunch(bs=bs, pad_first=pad_first, pad_idx=pad_idx))

In [ ]:
print('[CLS] token :', transformer_tokenizer.cls_token)
print('[SEP] token :', transformer_tokenizer.sep_token)
print('[PAD] token :', transformer_tokenizer.pad_token)
databunch.show_batch()

In [ ]:
print('[CLS] id :', transformer_tokenizer.cls_token_id)
print('[SEP] id :', transformer_tokenizer.sep_token_id)
print('[PAD] id :', pad_idx)
test_one_batch = databunch.one_batch()[0]
print('Batch shape : ',test_one_batch.shape)
print(test_one_batch)

In [ ]:
# defining our model architecture 
class CustomTransformerModel(nn.Module):
    def __init__(self, transformer_model: PreTrainedModel):
        super(CustomTransformerModel,self).__init__()
        self.transformer = transformer_model
        
    def forward(self, input_ids, attention_mask=None):
        
        # attention_mask
        # Mask to avoid performing attention on padding token indices.
        # Mask values selected in ``[0, 1]``:
        # ``1`` for tokens that are NOT MASKED, ``0`` for MASKED tokens.
        attention_mask = (input_ids!=pad_idx).type(input_ids.type()) 
        
        logits = self.transformer(input_ids,
                                  attention_mask = attention_mask)[0]   
        return logits

In [ ]:
config = config_class.from_pretrained(pretrained_model_name)
config.num_labels = 3
config.use_bfloat16 = use_fp16
print(config)

In [ ]:
transformer_model = model_class.from_pretrained(pretrained_model_name, config = config)
# transformer_model = model_class.from_pretrained(pretrained_model_name, num_labels = 5)

custom_transformer_model = CustomTransformerModel(transformer_model = transformer_model)

In [ ]:
from fastai.callbacks import *
from transformers import AdamW
from functools import partial

CustomAdamW = partial(AdamW, correct_bias=False)

learner = Learner(databunch, 
                  custom_transformer_model, 
                  opt_func = CustomAdamW, 
                  metrics=[accuracy, error_rate])

# Show graph of learner stats and metrics after each epoch.
learner.callbacks.append(ShowGraph(learner))

# Put learn in FP16 precision mode. --> Seems to not working
if use_fp16: learner = learner.to_fp16()

In [ ]:
print(learner.model)

In [ ]:
list_layers = [learner.model.transformer.roberta.embeddings,
              learner.model.transformer.roberta.encoder.layer[0],
              learner.model.transformer.roberta.encoder.layer[1],
              learner.model.transformer.roberta.encoder.layer[2],
              learner.model.transformer.roberta.encoder.layer[3],
              learner.model.transformer.roberta.encoder.layer[4],
              learner.model.transformer.roberta.encoder.layer[5],
              learner.model.transformer.roberta.encoder.layer[6],
              learner.model.transformer.roberta.encoder.layer[7],
              learner.model.transformer.roberta.encoder.layer[8],
              learner.model.transformer.roberta.pooler]

In [ ]:
learner.split(list_layers)
num_groups = len(learner.layer_groups)
print('Learner split in',num_groups,'groups')
print(learner.layer_groups)

In [ ]:
learner.save('untrain')

In [ ]:
seed_all(seed)
learner.load('untrain');

In [ ]:
learner.freeze_to(-1)

In [ ]:
learner.summary()

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(skip_end=10,suggestion=True)

In [ ]:
learner.fit_one_cycle(1,max_lr=1e-03)

In [ ]:
learner.save('first_cycle')

In [ ]:
seed_all(seed)
learner.load('first_cycle');

In [ ]:
learner.freeze_to(-2)

In [ ]:
lr = 1e-4

In [ ]:
learner.fit_one_cycle(1, max_lr=slice(lr*0.95**num_groups, lr), moms=(0.8, 0.9))

In [ ]:
learner.save('second_cycle')

In [ ]:
seed_all(seed)
learner.load('second_cycle');

In [ ]:
learner.freeze_to(-3)

In [ ]:
learner.fit_one_cycle(1, max_lr=slice(lr*0.95**num_groups, lr), moms=(0.8, 0.9))

In [ ]:
learner.save('third_cycle')

In [ ]:
seed_all(seed)
learner.load('third_cycle');

In [ ]:
learner.unfreeze()

In [ ]:
learner.fit_one_cycle(2, max_lr=slice(lr*0.95**num_groups, lr), moms=(0.8, 0.9))

In [ ]:
learner.predict('love friendship cool companiionship')

In [ ]:
learner.predict('kill hate fuck fire')

In [ ]:
learner.export(file = 'transformer.pkl');

In [ ]:
path = '/kaggle/working'
export_learner = load_learner(path, file = 'transformer.pkl')

In [ ]:
export_learner.predict('This is the worst movie of 2020')


In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learner.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in databunch.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    return preds[reverse_sampler, :]

test_preds = get_preds_as_nparray(DatasetType.Test)

In [ ]:
sample_submission = pd.DataFrame()
sample_submission['sentiment_class'] = np.argmax(test_preds,axis=1)
sample_submission.to_csv("predictions.csv", index=False)

In [ ]:
test.head()

In [ ]:
sample_submission['sentiment_class'].value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=2)
for train_index,test_index in skf.split(X,y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train_bert = X_train.original_text
    X_test_bert = X_test.original_text
    
    train_input = bert_encode(X_train_bert.values, tokenizer, max_len=140)
    test_input = bert_encode(X_test_bert.values, tokenizer, max_len=140)
    
    train_labels = y_train.values
    
    model = build_model(bert_layer, max_len=140)
    
    model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=1,
    batch_size=5
    )
    

In [ ]:
testing_input = bert_encode(test.original_text, tokenizer, max_len=140)

In [ ]:
prediction = model.predict(testing_input)
pred = []
for x in prediction:
    pred.append(int(x.round()))

In [ ]:
len(pred)

In [ ]:
test_id = pd.read_csv("/kaggle/input/hackerearth-tweet-sentiment/test.csv")
idd= test_id['id']
submission_2= pd.DataFrame(idd)

In [ ]:
submission_2['id']= submission_2['id'].astype(str)

In [ ]:
submission_2['sentiment_class'] = pred

In [ ]:
submission_2['sentiment_class'].value_counts()

In [ ]:
submission_2.head(2)

In [ ]:
submission_2.to_csv("bert.csv",index=False)